In [ ]:
# Definition of some PATHS
DRIVE_PATH = '/content/drive'

# Mounting Google Drive
import os
from google.colab import drive

if not os.path.isdir(DRIVE_PATH):
  drive.mount(DRIVE_PATH)

# Install dependencies from requirements.txt
!pip install -q -r /content/drive/MyDrive/Colab_Notebooks/requirements.txt

# Copy notebooks to root path to import them
!cp -r /content/drive/MyDrive/Colab_Notebooks/. /content/

# Unzip datasets in local folder
!unzip -q "/content/drive/MyDrive/Datasets/gsv_xs.zip" -d "/content/Datasets/"
!unzip -q "/content/drive/MyDrive/Datasets/sf_xs.zip" -d "/content/Datasets/"
!unzip -q "/content/drive/MyDrive/Datasets/tokyo_xs.zip" -d "/content/Datasets/"

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 109.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.4/111.4 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 119.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 56.4 MB/s eta 0:00:00


# MultiSimilarityMiner - MultiSimilarityLoss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='MultiSimilarityLoss',
    miner_name='MultiSimilarityMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

importing Jupyter notebook from GSVCitiesDataloader.ipynb
importing Jupyter notebook from GSVCitiesDataset.ipynb
importing Jupyter notebook from SanFranciscoDataset.ipynb
importing Jupyter notebook from TokyoDataset.ipynb
importing Jupyter notebook from VPRModel.ipynb
importing Jupyter notebook from Losses.ipynb


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1


importing Jupyter notebook from Validation.ipynb
importing Jupyter notebook from Helper.ipynb
importing Jupyter notebook from Resnet.ipynb
importing Jupyter notebook from gem.ipynb
importing Jupyter notebook from avg.ipynb


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)


+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                 | Params
----------------------------------------------------
0 | loss_fn    | MultiSimilarityLoss  | 0     
1 | miner      | MultiSimilarityMiner | 0     
2 | backbone   | ResNet               | 2.8 M 
3 | aggregator | GeMPool              | 1     
----------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 33.97 | 50.23 | 56.70 | 60.89 | 63.78 | 66.28 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 60.14 | 74.24 | 79.44 | 82.33 | 84.40 | 86.00 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 59.94 | 74.24 | 79.14 | 82.28 | 84.39 | 85.89 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 26min 5s, sys: 23min 13s, total: 1h 49min 18s
Wall time: 1h 53min 55s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_MULTISIMILARITYMINER_MSL.zip /content/LOGS

!cp /content/LOGS_GEM_MULTISIMILARITYMINER_MSL.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/events.out.tfevents.1720172490.0abbf8204c7b (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/hparams.yaml (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5994]_R5[0.7424].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(06)_step(2191)_R1[0.5875]_R5[0.7324].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6014]_R5[0.7424].ckpt (deflated 7%)


In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_MULTISIMILARITYMINER_MSL.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6014]_R5[0.7424].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6014]_R5[0.7424].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 25.10 | 40.30 | 47.60 | 50.60 | 52.40 | 54.40 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 37.46 | 58.41 | 66.03 | 72.38 | 76.51 | 79.05 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.251           │           0.251           │
│    SanFrancisco_xs/R10    │           0.476           │           0.476           │
│    SanFrancisco_xs/R5     │           0.403           │           0.403           │
│        Tokyo_xs/R1        │    0.3746031746031746     │    0.3746031746031746     │
│       Tokyo_xs/R10        │    0.6603174603174603     │    0.6603174603174603     │
│        Tokyo_xs/R5        │    0.5841269841269842     │    0.5841269841269842     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.1 s, sys: 21.5 s, total: 1min 7s
Wall time: 57.9 s


# NewMultiSimilarityMiner - MultiSimilarity Loss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='MultiSimilarityLoss',
    miner_name='NewMultiSimilarityMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                    | Params
-------------------------------------------------------
0 | loss_fn    | MultiSimilarityLoss     | 0     
1 | miner      | NewMultiSimilarityMiner | 0     
2 | backbone   | ResNet                  | 2.8 M 
3 | aggregator | GeMPool                 | 1     
-------------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 34.34 | 50.47 | 56.80 | 61.17 | 64.01 | 66.51 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 59.38 | 73.65 | 79.18 | 81.91 | 84.05 | 85.74 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 59.80 | 74.04 | 79.32 | 82.22 | 84.49 | 86.10 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 21min 58s, sys: 22min 52s, total: 1h 44min 51s
Wall time: 1h 51min 5s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_NEWMULTISIMILARITYMINER_MSL.zip /content/LOGS

!cp /content/LOGS_GEM_NEWMULTISIMILARITYMINER_MSL.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720179387.0abbf8204c7b (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5938]_R5[0.7365].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5980]_R5[0.7404].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(05)_step(1878)_R1[0.5945]_R5[0.7331].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/ (stored 0%)
  adding: conten

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_NEWMULTISIMILARITYMINER_MSL.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5980]_R5[0.7404].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5980]_R5[0.7404].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 24.20 | 37.70 | 45.40 | 49.60 | 52.10 | 54.30 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 34.29 | 52.06 | 60.00 | 65.08 | 70.79 | 74.60 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.242           │           0.242           │
│    SanFrancisco_xs/R10    │           0.454           │           0.454           │
│    SanFrancisco_xs/R5     │           0.377           │           0.377           │
│        Tokyo_xs/R1        │    0.34285714285714286    │    0.34285714285714286    │
│       Tokyo_xs/R10        │            0.6            │            0.6            │
│        Tokyo_xs/R5        │    0.5206349206349207     │    0.5206349206349207     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 45.8 s, sys: 20.6 s, total: 1min 6s
Wall time: 53.7 s


# TripletMarginMiner - MultiSimilarity Loss - 200 batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='MultiSimilarityLoss',
    miner_name='TripletMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                | Params
---------------------------------------------------
0 | loss_fn    | MultiSimilarityLoss | 0     
1 | miner      | TripletMarginMiner  | 0     
2 | backbone   | ResNet              | 2.8 M 
3 | aggregator | GeMPool             | 1     
---------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 24.77 | 38.57 | 45.30 | 49.59 | 52.48 | 55.24 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 58.84 | 73.29 | 78.57 | 81.82 | 83.55 | 85.10 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 58.65 | 73.43 | 78.73 | 81.58 | 83.57 | 85.07 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 24min 41s, sys: 22min 54s, total: 1h 47min 36s
Wall time: 1h 52min 6s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_TRIPLETMARGINMINER_MSL.zip /content/LOGS

!cp /content/LOGS_GEM_TRIPLETMARGINMINER_MSL.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720186053.0abbf8204c7b (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720179387.0abbf8204c7b (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5938]_R5[0.7365].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5980]_R5[0.7404].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(05)_step(1878)_R1[0.5945]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_TRIPLETMARGINMINER_MSL.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_2/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5884]_R5[0.7329].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_2/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5884]_R5[0.7329].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 21.10 | 36.50 | 41.60 | 45.80 | 48.80 | 50.50 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 32.06 | 58.10 | 64.44 | 69.84 | 72.70 | 75.56 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.211           │           0.211           │
│    SanFrancisco_xs/R10    │           0.416           │           0.416           │
│    SanFrancisco_xs/R5     │           0.365           │           0.365           │
│        Tokyo_xs/R1        │    0.32063492063492066    │    0.32063492063492066    │
│       Tokyo_xs/R10        │    0.6444444444444445     │    0.6444444444444445     │
│        Tokyo_xs/R5        │     0.580952380952381     │     0.580952380952381     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 45.8 s, sys: 21.5 s, total: 1min 7s
Wall time: 53.8 s


# NewTripletMarginMiner - MultiSimilarity Loss - 200 batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='MultiSimilarityLoss',
    miner_name='NewTripletMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                  | Params
-----------------------------------------------------
0 | loss_fn    | MultiSimilarityLoss   | 0     
1 | miner      | NewTripletMarginMiner | 0     
2 | backbone   | ResNet                | 2.8 M 
3 | aggregator | GeMPool               | 1     
-----------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 24.76 | 38.58 | 45.39 | 49.59 | 52.63 | 55.37 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 59.84 | 74.04 | 79.24 | 82.57 | 84.74 | 86.28 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 59.99 | 74.42 | 79.72 | 82.33 | 84.32 | 86.11 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 25min 2s, sys: 22min 47s, total: 1h 47min 50s
Wall time: 1h 52min 41s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_NEWTRIPLETMARGINMINER_MSL.zip /content/LOGS

!cp /content/LOGS_GEM_NEWTRIPLETMARGINMINER_MSL.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720186053.0abbf8204c7b (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720179387.0abbf8204c7b (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5938]_R5[0.7365].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5980]_R5[0.7404].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(05)_step(1878)_R1[0.5945]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_NEWTRIPLETMARGINMINER_MSL.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_3/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5999]_R5[0.7442].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_3/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5999]_R5[0.7442].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 23.60 | 38.50 | 45.00 | 48.80 | 51.00 | 53.30 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 36.19 | 57.46 | 65.08 | 68.57 | 71.43 | 75.24 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.236           │           0.236           │
│    SanFrancisco_xs/R10    │           0.45            │           0.45            │
│    SanFrancisco_xs/R5     │           0.385           │           0.385           │
│        Tokyo_xs/R1        │    0.3619047619047619     │    0.3619047619047619     │
│       Tokyo_xs/R10        │    0.6507936507936508     │    0.6507936507936508     │
│        Tokyo_xs/R5        │    0.5746031746031746     │    0.5746031746031746     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.1 s, sys: 21.3 s, total: 1min 7s
Wall time: 52.4 s


# PairMarginMiner - MultiSimilarity Loss - 200 batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='MultiSimilarityLoss',
    miner_name='PairMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_PAIRMARGINMINER_MSL.zip /content/LOGS

!cp /content/LOGS_GEM_PAIRMARGINMINER_MSL.zip /content/drive/MyDrive/

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_PAIRMARGINMINER_MSL.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_4/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6043]_R5[0.7523].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_4/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6043]_R5[0.7523].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 26.30 | 41.30 | 47.40 | 51.60 | 54.00 | 55.80 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 39.05 | 59.05 | 64.76 | 70.16 | 73.33 | 75.56 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.263           │           0.263           │
│    SanFrancisco_xs/R10    │           0.474           │           0.474           │
│    SanFrancisco_xs/R5     │           0.413           │           0.413           │
│        Tokyo_xs/R1        │    0.3904761904761905     │    0.3904761904761905     │
│       Tokyo_xs/R10        │    0.6476190476190476     │    0.6476190476190476     │
│        Tokyo_xs/R5        │    0.5904761904761905     │    0.5904761904761905     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 45.6 s, sys: 21.1 s, total: 1min 6s
Wall time: 54.1 s


# NewPairMarginMiner - MultiSimilarity Loss - 200 batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='MultiSimilarityLoss',
    miner_name='NewPairMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

importing Jupyter notebook from GSVCitiesDataloader.ipynb
importing Jupyter notebook from GSVCitiesDataset.ipynb
importing Jupyter notebook from SanFranciscoDataset.ipynb
importing Jupyter notebook from TokyoDataset.ipynb
importing Jupyter notebook from VPRModel.ipynb
importing Jupyter notebook from Losses.ipynb


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1


importing Jupyter notebook from Validation.ipynb
importing Jupyter notebook from Helper.ipynb
importing Jupyter notebook from Resnet.ipynb
importing Jupyter notebook from gem.ipynb
importing Jupyter notebook from avg.ipynb


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)


+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                | Params
---------------------------------------------------
0 | loss_fn    | MultiSimilarityLoss | 0     
1 | miner      | NewPairMarginMiner  | 0     
2 | backbone   | ResNet              | 2.8 M 
3 | aggregator | GeMPool             | 1     
---------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 16.00 | 25.42 | 31.73 | 35.42 | 38.30 | 40.42 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 59.86 | 74.14 | 79.92 | 82.56 | 84.49 | 85.85 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 60.99 | 75.15 | 80.51 | 83.16 | 85.04 | 86.44 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 28min 27s, sys: 23min 20s, total: 1h 51min 47s
Wall time: 1h 52min 47s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_NEWPAIRMARGINMINER_MSL.zip /content/LOGS

!cp /content/LOGS_GEM_NEWPAIRMARGINMINER_MSL.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.6087]_R5[0.7484].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6099]_R5[0.7515].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5986]_R5[0.7414].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/events.out.tfevents.1720251586.c115c5ef8723 (deflated 64%)


In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_NEWPAIRMARGINMINER_MSL.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6099]_R5[0.7515].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6099]_R5[0.7515].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 26.30 | 40.30 | 46.90 | 51.40 | 53.50 | 55.60 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 42.54 | 61.90 | 71.11 | 73.65 | 75.56 | 77.78 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.263           │           0.263           │
│    SanFrancisco_xs/R10    │           0.469           │           0.469           │
│    SanFrancisco_xs/R5     │           0.403           │           0.403           │
│        Tokyo_xs/R1        │    0.4253968253968254     │    0.4253968253968254     │
│       Tokyo_xs/R10        │    0.7111111111111111     │    0.7111111111111111     │
│        Tokyo_xs/R5        │    0.6190476190476191     │    0.6190476190476191     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.4 s, sys: 20.6 s, total: 1min 7s
Wall time: 58 s


# MultiSimiliratyMiner - TripletMarginLoss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='TripletMarginLoss',
    miner_name='MultiSimilarityMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                 | Params
----------------------------------------------------
0 | loss_fn    | TripletMarginLoss    | 0     
1 | miner      | MultiSimilarityMiner | 0     
2 | backbone   | ResNet               | 2.8 M 
3 | aggregator | GeMPool              | 1     
----------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 48.77 | 62.89 | 68.89 | 72.46 | 75.15 | 77.28 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 64.94 | 77.98 | 82.76 | 85.54 | 87.11 | 88.20 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 64.86 | 78.09 | 82.81 | 85.51 | 87.21 | 88.48 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 24min 36s, sys: 23min 18s, total: 1h 47min 54s
Wall time: 1h 53min 11s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_MULTISIMILARITYMINER_TRIPLETLOSS.zip /content/LOGS

!cp /content/LOGS_GEM_MULTISIMILARITYMINER_TRIPLETLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720258415.c115c5ef8723 (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(06)_step(2191)_R1[0.6491]_R5[0.7751].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6494]_R5[0.7798].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6486]_R5[0.7809].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/ (stored 0%)
  adding: conten

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_MULTISIMILARITYMINER_TRIPLETLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6494]_R5[0.7798].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6494]_R5[0.7798].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 26.60 | 39.60 | 46.30 | 50.30 | 53.80 | 56.50 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 37.78 | 56.51 | 64.76 | 68.89 | 73.65 | 78.10 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.266           │           0.266           │
│    SanFrancisco_xs/R10    │           0.463           │           0.463           │
│    SanFrancisco_xs/R5     │           0.396           │           0.396           │
│        Tokyo_xs/R1        │    0.37777777777777777    │    0.37777777777777777    │
│       Tokyo_xs/R10        │    0.6476190476190476     │    0.6476190476190476     │
│        Tokyo_xs/R5        │    0.5650793650793651     │    0.5650793650793651     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 45.6 s, sys: 21.2 s, total: 1min 6s
Wall time: 55.6 s


# NewMultiSimiliratyMiner - TripletMarginLoss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='TripletMarginLoss',
    miner_name='NewMultiSimilarityMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                    | Params
-------------------------------------------------------
0 | loss_fn    | TripletMarginLoss       | 0     
1 | miner      | NewMultiSimilarityMiner | 0     
2 | backbone   | ResNet                  | 2.8 M 
3 | aggregator | GeMPool                 | 1     
-------------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 48.77 | 62.88 | 68.91 | 72.50 | 75.13 | 77.27 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 64.73 | 78.04 | 82.66 | 85.40 | 87.04 | 88.26 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 64.88 | 78.04 | 82.67 | 85.55 | 87.18 | 88.39 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 24min 11s, sys: 23min 1s, total: 1h 47min 13s
Wall time: 1h 52min 23s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_NEWMULTISIMILIRATYMINER_TRIPLETLOSS.zip /content/LOGS

!cp /content/LOGS_GEM_NEWMULTISIMILIRATYMINER_TRIPLETLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720265207.c115c5ef8723 (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720258415.c115c5ef8723 (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(06)_step(2191)_R1[0.6491]_R5[0.7751].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6494]_R5[0.7798].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6486]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_NEWMULTISIMILIRATYMINER_TRIPLETLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_2/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6488]_R5[0.7804].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_2/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6488]_R5[0.7804].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 26.50 | 40.50 | 47.50 | 51.90 | 54.10 | 56.80 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 40.32 | 56.83 | 64.44 | 69.52 | 72.38 | 75.24 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.265           │           0.265           │
│    SanFrancisco_xs/R10    │           0.475           │           0.475           │
│    SanFrancisco_xs/R5     │           0.405           │           0.405           │
│        Tokyo_xs/R1        │    0.4031746031746032     │    0.4031746031746032     │
│       Tokyo_xs/R10        │    0.6444444444444445     │    0.6444444444444445     │
│        Tokyo_xs/R5        │    0.5682539682539682     │    0.5682539682539682     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.1 s, sys: 21.4 s, total: 1min 7s
Wall time: 55.5 s


# TripletMarginMiner - TripletMarginLoss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='TripletMarginLoss',
    miner_name='TripletMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type               | Params
--------------------------------------------------
0 | loss_fn    | TripletMarginLoss  | 0     
1 | miner      | TripletMarginMiner | 0     
2 | backbone   | ResNet             | 2.8 M 
3 | aggregator | GeMPool            | 1     
--------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 44.68 | 59.40 | 65.41 | 69.32 | 72.26 | 74.30 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 62.37 | 76.12 | 81.15 | 83.91 | 85.69 | 87.29 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 62.53 | 76.23 | 81.45 | 84.20 | 86.09 | 87.58 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 27min 19s, sys: 23min 3s, total: 1h 50min 22s
Wall time: 1h 53min 52s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_TRIPLETMARGINMINER_TRIPLETLOSS.zip /content/LOGS

!cp /content/LOGS_GEM_TRIPLETMARGINMINER_TRIPLETLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720265207.c115c5ef8723 (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720258415.c115c5ef8723 (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(06)_step(2191)_R1[0.6491]_R5[0.7751].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6494]_R5[0.7798].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6486]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_TRIPLETMARGINMINER_TRIPLETLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_3/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6253]_R5[0.7623].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_3/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6253]_R5[0.7623].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 23.80 | 37.90 | 43.60 | 48.40 | 51.10 | 53.00 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 36.19 | 53.97 | 65.08 | 69.52 | 73.97 | 76.51 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.238           │           0.238           │
│    SanFrancisco_xs/R10    │           0.436           │           0.436           │
│    SanFrancisco_xs/R5     │           0.379           │           0.379           │
│        Tokyo_xs/R1        │    0.3619047619047619     │    0.3619047619047619     │
│       Tokyo_xs/R10        │    0.6507936507936508     │    0.6507936507936508     │
│        Tokyo_xs/R5        │    0.5396825396825397     │    0.5396825396825397     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.6 s, sys: 22.3 s, total: 1min 8s
Wall time: 56.2 s


# NewTripletMarginMiner - TripletMarginLoss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='TripletMarginLoss',
    miner_name='NewTripletMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                  | Params
-----------------------------------------------------
0 | loss_fn    | TripletMarginLoss     | 0     
1 | miner      | NewTripletMarginMiner | 0     
2 | backbone   | ResNet                | 2.8 M 
3 | aggregator | GeMPool               | 1     
-----------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 44.65 | 59.30 | 65.34 | 69.25 | 72.11 | 74.24 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 62.04 | 75.77 | 81.02 | 83.66 | 85.54 | 86.90 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 61.09 | 75.42 | 80.65 | 83.24 | 85.12 | 86.58 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 27min 9s, sys: 23min 47s, total: 1h 50min 57s
Wall time: 1h 56min 5s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_NEWTRIPLETMARGINMINER_TRIPLETLOSS.zip /content/LOGS

!cp /content/LOGS_GEM_NEWTRIPLETMARGINMINER_TRIPLETLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720265207.c115c5ef8723 (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720258415.c115c5ef8723 (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(06)_step(2191)_R1[0.6491]_R5[0.7751].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6494]_R5[0.7798].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6486]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_NEWTRIPLETMARGINMINER_TRIPLETLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_4/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6204]_R5[0.7577].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_4/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6204]_R5[0.7577].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 22.60 | 36.90 | 43.20 | 47.00 | 49.80 | 52.30 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 33.97 | 53.97 | 61.59 | 67.30 | 71.11 | 74.29 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.226           │           0.226           │
│    SanFrancisco_xs/R10    │           0.432           │           0.432           │
│    SanFrancisco_xs/R5     │           0.369           │           0.369           │
│        Tokyo_xs/R1        │    0.3396825396825397     │    0.3396825396825397     │
│       Tokyo_xs/R10        │    0.6158730158730159     │    0.6158730158730159     │
│        Tokyo_xs/R5        │    0.5396825396825397     │    0.5396825396825397     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 47 s, sys: 22.2 s, total: 1min 9s
Wall time: 57.7 s


# PairMarginMiner - TripletMarginLoss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='TripletMarginLoss',
    miner_name='PairMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type              | Params
-------------------------------------------------
0 | loss_fn    | TripletMarginLoss | 0     
1 | miner      | PairMarginMiner   | 0     
2 | backbone   | ResNet            | 2.8 M 
3 | aggregator | GeMPool           | 1     
-------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 20.77 | 31.04 | 35.61 | 39.31 | 41.81 | 43.71 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 20.64 | 30.95 | 35.42 | 39.21 | 41.70 | 43.69 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 20.81 | 31.16 | 35.63 | 39.43 | 41.94 | 43.83 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 19min 7s, sys: 23min 32s, total: 1h 42min 39s
Wall time: 1h 54min 41s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_PAIRMARGINMINER_TRIPLETLOSS.zip /content/LOGS

!cp /content/LOGS_GEM_PAIRMARGINMINER_TRIPLETLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/events.out.tfevents.1720286000.c115c5ef8723 (deflated 68%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(05)_step(1878)_R1[0.2081]_R5[0.3119].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(01)_step(0626)_R1[0.2078]_R5[0.3100].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.2081]_R5[0.3116].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: conten

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_PAIRMARGINMINER_TRIPLETLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(05)_step(1878)_R1[0.2081]_R5[0.3119].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(05)_step(1878)_R1[0.2081]_R5[0.3119].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+---------------------------------------------------------+
|           Performance on Test SanFrancisco_xs           |
+----------+------+-------+-------+-------+-------+-------+
|    K     |  1   |   5   |   10  |   15  |   20  |   25  |
+----------+------+-------+-------+-------+-------+-------+
| Recall@K | 5.70 | 14.70 | 19.40 | 22.50 | 24.90 | 26.50 |
+----------+------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 16.51 | 30.79 | 40.00 | 47.62 | 53.33 | 56.19 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.057           │           0.057           │
│    SanFrancisco_xs/R10    │           0.194           │           0.194           │
│    SanFrancisco_xs/R5     │           0.147           │           0.147           │
│        Tokyo_xs/R1        │    0.16507936507936508    │    0.16507936507936508    │
│       Tokyo_xs/R10        │            0.4            │            0.4            │
│        Tokyo_xs/R5        │    0.30793650793650795    │    0.30793650793650795    │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.4 s, sys: 21.7 s, total: 1min 8s
Wall time: 54.2 s


# NewPairMarginMiner - TripletMarginLoss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='TripletMarginLoss',
    miner_name='NewPairMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type               | Params
--------------------------------------------------
0 | loss_fn    | TripletMarginLoss  | 0     
1 | miner      | NewPairMarginMiner | 0     
2 | backbone   | ResNet             | 2.8 M 
3 | aggregator | GeMPool            | 1     
--------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 20.77 | 31.04 | 35.61 | 39.31 | 41.81 | 43.71 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 20.64 | 30.95 | 35.42 | 39.21 | 41.70 | 43.69 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 20.81 | 31.16 | 35.63 | 39.43 | 41.94 | 43.83 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 18min 31s, sys: 23min 55s, total: 1h 42min 27s
Wall time: 1h 57min 6s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_NEWPAIRMARGINMINER_TRIPLETLOSS.zip /content/LOGS

!cp /content/LOGS_GEM_NEWPAIRMARGINMINER_TRIPLETLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/events.out.tfevents.1720286000.c115c5ef8723 (deflated 68%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/events.out.tfevents.1720292890.c115c5ef8723 (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(05)_step(1878)_R1[0.2081]_R5[0.3119].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(01)_step(0626)_R1[0.2078]_R5[0.3100].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.2081]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_NEWPAIRMARGINMINER_TRIPLETLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_6/checkpoints/resnet18_epoch(05)_step(1878)_R1[0.2081]_R5[0.3119].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_6/checkpoints/resnet18_epoch(05)_step(1878)_R1[0.2081]_R5[0.3119].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+---------------------------------------------------------+
|           Performance on Test SanFrancisco_xs           |
+----------+------+-------+-------+-------+-------+-------+
|    K     |  1   |   5   |   10  |   15  |   20  |   25  |
+----------+------+-------+-------+-------+-------+-------+
| Recall@K | 5.70 | 14.70 | 19.40 | 22.50 | 24.90 | 26.50 |
+----------+------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 16.51 | 30.79 | 40.00 | 47.62 | 53.33 | 56.19 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.057           │           0.057           │
│    SanFrancisco_xs/R10    │           0.194           │           0.194           │
│    SanFrancisco_xs/R5     │           0.147           │           0.147           │
│        Tokyo_xs/R1        │    0.16507936507936508    │    0.16507936507936508    │
│       Tokyo_xs/R10        │            0.4            │            0.4            │
│        Tokyo_xs/R5        │    0.30793650793650795    │    0.30793650793650795    │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.1 s, sys: 23 s, total: 1min 9s
Wall time: 58.7 s


# MultiSimiliratyMiner - FastAPLoss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='FastAPLoss',
    miner_name='MultiSimilarityMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                 | Params
----------------------------------------------------
0 | loss_fn    | FastAPLoss           | 0     
1 | miner      | MultiSimilarityMiner | 0     
2 | backbone   | ResNet               | 2.8 M 
3 | aggregator | GeMPool              | 1     
----------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 30.30 | 45.00 | 52.18 | 56.82 | 59.59 | 62.15 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 66.32 | 78.26 | 82.86 | 85.20 | 86.78 | 88.08 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 66.38 | 78.19 | 82.48 | 85.06 | 86.50 | 87.93 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 24min 36s, sys: 23min 41s, total: 1h 48min 18s
Wall time: 1h 56min 3s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_MULTISIMILARITYMINER_FASTAPLOSS.zip /content/LOGS

!cp /content/LOGS_GEM_MULTISIMILARITYMINER_FASTAPLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/events.out.tfevents.1720286000.c115c5ef8723 (deflated 68%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/events.out.tfevents.1720292890.c115c5ef8723 (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(05)_step(1878)_R1[0.2081]_R5[0.3119].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(01)_step(0626)_R1[0.2078]_R5[0.3100].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.2081]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_MULTISIMILARITYMINER_FASTAPLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_7/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6638]_R5[0.7819].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_7/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6638]_R5[0.7819].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 31.50 | 45.30 | 51.20 | 55.70 | 58.60 | 61.80 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 48.89 | 63.49 | 69.84 | 75.87 | 78.41 | 80.32 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.315           │           0.315           │
│    SanFrancisco_xs/R10    │           0.512           │           0.512           │
│    SanFrancisco_xs/R5     │           0.453           │           0.453           │
│        Tokyo_xs/R1        │    0.4888888888888889     │    0.4888888888888889     │
│       Tokyo_xs/R10        │    0.6984126984126984     │    0.6984126984126984     │
│        Tokyo_xs/R5        │    0.6349206349206349     │    0.6349206349206349     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.5 s, sys: 21.5 s, total: 1min 7s
Wall time: 55.8 s


# MultiSimilarityMiner - FastAP Loss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='FastAPLoss',
    miner_name='NewMultiSimilarityMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

importing Jupyter notebook from GSVCitiesDataloader.ipynb
importing Jupyter notebook from GSVCitiesDataset.ipynb
importing Jupyter notebook from SanFranciscoDataset.ipynb
importing Jupyter notebook from TokyoDataset.ipynb
importing Jupyter notebook from VPRModel.ipynb
importing Jupyter notebook from Losses.ipynb


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1


importing Jupyter notebook from Validation.ipynb
importing Jupyter notebook from Helper.ipynb
importing Jupyter notebook from Resnet.ipynb
importing Jupyter notebook from gem.ipynb
importing Jupyter notebook from avg.ipynb


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)


+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                    | Params
-------------------------------------------------------
0 | loss_fn    | FastAPLoss              | 0     
1 | miner      | NewMultiSimilarityMiner | 0     
2 | backbone   | ResNet                  | 2.8 M 
3 | aggregator | GeMPool                 | 1     
-------------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 30.11 | 44.86 | 51.82 | 56.16 | 59.44 | 62.02 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 66.51 | 78.28 | 82.52 | 85.21 | 86.84 | 88.04 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 66.21 | 78.31 | 82.75 | 85.14 | 86.99 | 88.16 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 23min 29s, sys: 24min 8s, total: 1h 47min 37s
Wall time: 1h 55min 39s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_NEWMULTISIMILARITYMINER_FASTAPLOSS.zip /content/LOGS

!cp /content/LOGS_GEM_NEWMULTISIMILARITYMINER_FASTAPLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/events.out.tfevents.1720344554.b18cbecd3081 (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.6577]_R5[0.7774].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6651]_R5[0.7828].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6621]_R5[0.7831].ckpt (deflated 7%)


In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_NEWMULTISIMILARITYMINER_FASTAPLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6651]_R5[0.7828].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6651]_R5[0.7828].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 33.60 | 47.30 | 53.50 | 56.80 | 59.50 | 61.40 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 45.71 | 65.08 | 72.38 | 75.24 | 77.14 | 79.68 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.336           │           0.336           │
│    SanFrancisco_xs/R10    │           0.535           │           0.535           │
│    SanFrancisco_xs/R5     │           0.473           │           0.473           │
│        Tokyo_xs/R1        │    0.45714285714285713    │    0.45714285714285713    │
│       Tokyo_xs/R10        │    0.7238095238095238     │    0.7238095238095238     │
│        Tokyo_xs/R5        │    0.6507936507936508     │    0.6507936507936508     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 45.4 s, sys: 20.8 s, total: 1min 6s
Wall time: 57 s


# TripletMarginMiner - FastAP Loss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='FastAPLoss',
    miner_name='TripletMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type               | Params
--------------------------------------------------
0 | loss_fn    | FastAPLoss         | 0     
1 | miner      | TripletMarginMiner | 0     
2 | backbone   | ResNet             | 2.8 M 
3 | aggregator | GeMPool            | 1     
--------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 31.21 | 46.19 | 53.27 | 57.59 | 60.55 | 63.28 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 65.96 | 78.03 | 82.33 | 84.77 | 86.61 | 87.85 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 65.73 | 78.14 | 82.85 | 84.94 | 86.73 | 87.93 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 22min 54s, sys: 23min 52s, total: 1h 46min 46s
Wall time: 1h 56min


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_TRIPLETMARGINMINER_FASTAPLOSS.zip /content/LOGS

!cp /content/LOGS_GEM_TRIPLETMARGINMINER_FASTAPLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720351551.b18cbecd3081 (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.6493]_R5[0.7721].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6573]_R5[0.7814].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6596]_R5[0.7803].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/ (stored 0%)
  adding: conten

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_TRIPLETMARGINMINER_FASTAPLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6596]_R5[0.7803].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6596]_R5[0.7803].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 30.70 | 45.60 | 51.50 | 56.40 | 59.50 | 62.00 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 46.03 | 65.40 | 73.65 | 77.78 | 79.68 | 81.90 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.307           │           0.307           │
│    SanFrancisco_xs/R10    │           0.515           │           0.515           │
│    SanFrancisco_xs/R5     │           0.456           │           0.456           │
│        Tokyo_xs/R1        │    0.4603174603174603     │    0.4603174603174603     │
│       Tokyo_xs/R10        │    0.7365079365079366     │    0.7365079365079366     │
│        Tokyo_xs/R5        │     0.653968253968254     │     0.653968253968254     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.1 s, sys: 22.6 s, total: 1min 8s
Wall time: 57.5 s


# NewTripletMarginMiner - FastAP Loss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='FastAPLoss',
    miner_name='NewTripletMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                  | Params
-----------------------------------------------------
0 | loss_fn    | FastAPLoss            | 0     
1 | miner      | NewTripletMarginMiner | 0     
2 | backbone   | ResNet                | 2.8 M 
3 | aggregator | GeMPool               | 1     
-----------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 30.78 | 45.84 | 53.38 | 57.74 | 61.03 | 63.38 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 65.57 | 77.86 | 82.52 | 84.70 | 86.16 | 87.55 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 65.32 | 77.88 | 82.46 | 84.92 | 86.56 | 87.79 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 24min 6s, sys: 23min 58s, total: 1h 48min 5s
Wall time: 1h 57min 10s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_NEWTRIPLETMARGINMINER_FASTAPLOSS.zip /content/LOGS

!cp /content/LOGS_GEM_NEWTRIPLETMARGINMINER_FASTAPLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720351551.b18cbecd3081 (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720358513.b18cbecd3081 (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.6493]_R5[0.7721].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6573]_R5[0.7814].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6596]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_NEWTRIPLETMARGINMINER_FASTAPLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_2/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6557]_R5[0.7786].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_2/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6557]_R5[0.7786].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 29.50 | 45.70 | 51.10 | 54.80 | 58.50 | 61.00 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 49.52 | 66.35 | 72.38 | 77.14 | 78.73 | 80.00 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.295           │           0.295           │
│    SanFrancisco_xs/R10    │           0.511           │           0.511           │
│    SanFrancisco_xs/R5     │           0.457           │           0.457           │
│        Tokyo_xs/R1        │    0.49523809523809526    │    0.49523809523809526    │
│       Tokyo_xs/R10        │    0.7238095238095238     │    0.7238095238095238     │
│        Tokyo_xs/R5        │    0.6634920634920635     │    0.6634920634920635     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.7 s, sys: 22.3 s, total: 1min 9s
Wall time: 57.4 s


# PairMarginMiner - FastAP Loss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='FastAPLoss',
    miner_name='PairMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type            | Params
-----------------------------------------------
0 | loss_fn    | FastAPLoss      | 0     
1 | miner      | PairMarginMiner | 0     
2 | backbone   | ResNet          | 2.8 M 
3 | aggregator | GeMPool         | 1     
-----------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 31.67 | 46.15 | 53.43 | 57.74 | 61.34 | 63.98 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 65.72 | 77.63 | 82.01 | 84.56 | 86.45 | 87.74 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 66.10 | 78.08 | 82.13 | 84.77 | 86.18 | 87.64 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 23min 28s, sys: 24min 7s, total: 1h 47min 35s
Wall time: 1h 58min 23s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_PAIRMARGINMINER_FASTAPLOSS.zip /content/LOGS

!cp /content/LOGS_GEM_PAIRMARGINMINER_FASTAPLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720351551.b18cbecd3081 (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720358513.b18cbecd3081 (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.6493]_R5[0.7721].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6573]_R5[0.7814].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6596]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_PAIRMARGINMINER_FASTAPLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_3/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6610]_R5[0.7808].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_3/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6610]_R5[0.7808].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 31.70 | 46.70 | 53.00 | 57.20 | 59.70 | 61.70 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 45.71 | 65.71 | 73.97 | 78.73 | 80.32 | 82.22 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.317           │           0.317           │
│    SanFrancisco_xs/R10    │           0.53            │           0.53            │
│    SanFrancisco_xs/R5     │           0.467           │           0.467           │
│        Tokyo_xs/R1        │    0.45714285714285713    │    0.45714285714285713    │
│       Tokyo_xs/R10        │    0.7396825396825397     │    0.7396825396825397     │
│        Tokyo_xs/R5        │    0.6571428571428571     │    0.6571428571428571     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46 s, sys: 22.5 s, total: 1min 8s
Wall time: 58.8 s


# NewPairMarginMiner - FastAP Loss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # SanFrancisco_val
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, sf_xs and tokyo_xs available for testing
)

# examples of backbones
# resnet18, resnet50, resnet101, resnet152,
# resnext50_32x4d, resnext50_32x4d_swsl , resnext101_32x4d_swsl, resnext101_32x8d_swsl
# efficientnet_b0, efficientnet_b1, efficientnet_b2
# swinv2_base_window12to16_192to256_22kft1k

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='GeM',
    agg_config={'p': 3},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='FastAPLoss',
    miner_name='NewPairMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on pittsburg val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type               | Params
--------------------------------------------------
0 | loss_fn    | FastAPLoss         | 0     
1 | miner      | NewPairMarginMiner | 0     
2 | backbone   | ResNet             | 2.8 M 
3 | aggregator | GeMPool            | 1     
--------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 30.86 | 45.38 | 52.78 | 57.18 | 60.64 | 63.51 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 65.22 | 77.46 | 81.97 | 84.49 | 86.20 | 87.50 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 66.10 | 77.89 | 82.47 | 84.61 | 86.44 | 87.86 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 23min 6s, sys: 23min 49s, total: 1h 46min 56s
Wall time: 1h 57min 29s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_GEM_NEWPAIRMARGINMINER_FASTAPLOSS.zip /content/LOGS

!cp /content/LOGS_GEM_NEWPAIRMARGINMINER_FASTAPLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720351551.b18cbecd3081 (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720358513.b18cbecd3081 (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.6493]_R5[0.7721].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6573]_R5[0.7814].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6596]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

prediction_writer = BasePredictionWriter(write_interval='epoch')

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    callbacks=[prediction_writer],  # we run the checkpointing callback (you can add more)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_GEM_NEWPAIRMARGINMINER_FASTAPLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_4/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6610]_R5[0.7789].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_4/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6610]_R5[0.7789].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 32.10 | 47.90 | 54.40 | 57.70 | 60.90 | 62.50 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 43.49 | 66.03 | 73.97 | 76.19 | 80.63 | 82.22 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.321           │           0.321           │
│    SanFrancisco_xs/R10    │           0.544           │           0.544           │
│    SanFrancisco_xs/R5     │           0.479           │           0.479           │
│        Tokyo_xs/R1        │    0.43492063492063493    │    0.43492063492063493    │
│       Tokyo_xs/R10        │    0.7396825396825397     │    0.7396825396825397     │
│        Tokyo_xs/R5        │    0.6603174603174603     │    0.6603174603174603     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.1 s, sys: 22.8 s, total: 1min 8s
Wall time: 58.7 s


# VISUALIZATIONS

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

name = 'SanFrancisco_xs'
dr_name = 'sf_xs'
preds = model.custom_prediction(name)

test_ds = model.trainer.datamodule.test_datasets[model.trainer.datamodule.test_set_names.index(name)]

query_index = 100

img = mpimg.imread('/content/Datasets/'+ dr_name +'/test/queries/'+test_ds.qImages[query_index])
plt.imshow(img)
plt.axis('off')
plt.title('Query')
plt.show()

fig = plt.figure(figsize=(25, 25))
for i in range(0,25):
  img2 = mpimg.imread('/content/Datasets/'+ dr_name +'/test/database/'+test_ds.dbImages[int(preds[query_index,i])])
  rep = fig.add_subplot(5,5,i+1)
  rep.set_title('Prediction #' + str(i+1))
  if np.isin(preds[query_index], test_ds.ground_truth[query_index])[i]:
    rep.title.set_color('green')
    rep.title.set_fontweight('demibold')
  plt.imshow(img2)
  plt.axis('off')

plt.show()




Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import numpy as np

print(test_ds.ground_truth[query_index])
print('\n')
print(preds[query_index])
print('\n')

np.isin(preds[query_index], test_ds.ground_truth[query_index])

[2969, 2970, 2972, 2973, 2975, 2976, 2980, 2990, 2991, 2997]


tensor([ 2976,  2972,  2973,  2969,  5325,  4784,  1918, 14382, 12056, 17144,
        17096,  2631, 13782,  2970,   626,   961,  1990,   741, 18837,  2578,
         5404,  1543,   930, 16025,   954])




array([ True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False])